In [1]:
import bw2analyzer as bwa
import bw2calc as bc
import bw2data as bd
import bw2io as bi
#import ipywidgets as widgets
import pandas as pd
from bw2data.query import Filter, Query
from IPython.display import display

/Users/andreavargasf/anaconda3/envs/STI_environment/lib/python3.11/site-packages/bw2calc/__init__.py:48: UserWarning: 
It seems like you have an ARM architecture, but haven't installed scikit-umfpack:

    https://pypi.org/project/scikit-umfpack/

Installing it could give you much faster calculations.

  warnings.warn(UMFPACK_WARNING)


In [2]:
#Importing the variables with the project name and background db
from project_details import ei_name, project_name

In [3]:
bd.projects.set_current(project_name)
bd.databases

Databases dictionary with 3 object(s):
	asphalt
	biosphere3
	ecoinvent-391-cutoff

In [4]:
# Is the background database name the same as the one we wrote in `project_details.py`?
assert ei_name in bd.databases

In [5]:
fg_name = "asphalt"

In [6]:
fgdb = bd.Database(fg_name)

## System definition

In [7]:
pavement_structure = fgdb.get("pavement_structure")

In [9]:
bwa.print_recursive_supply_chain(pavement_structure, max_level=1)

1: 'pavement structure' (kilometer, NL, None)
  2.01e+06: 'AC Surf' (kilogram, NL, None)
  2.72e+06: 'AC Bin' (kilogram, NL, None)
  1.35e+07: 'AC Bin' (kilogram, NL, None)


## Functional unit 

In [26]:
fu = {pavement_structure:1}
fu

{'pavement structure' (kilometer, NL, None): 1}

## LCIA Methods

In [20]:
cml_methods = [m for m in bd.methods if m[0] == "CML v4.8 2016"]
cml_methods

[('CML v4.8 2016',
  'acidification',
  'acidification (incl. fate, average Europe total, A&B)'),
 ('CML v4.8 2016', 'climate change', 'global warming potential (GWP100)'),
 ('CML v4.8 2016',
  'ecotoxicity: freshwater',
  'freshwater aquatic ecotoxicity (FAETP inf)'),
 ('CML v4.8 2016',
  'ecotoxicity: marine',
  'marine aquatic ecotoxicity (MAETP inf)'),
 ('CML v4.8 2016',
  'ecotoxicity: terrestrial',
  'terrestrial ecotoxicity (TETP inf)'),
 ('CML v4.8 2016',
  'energy resources: non-renewable',
  'abiotic depletion potential (ADP): fossil fuels'),
 ('CML v4.8 2016', 'eutrophication', 'eutrophication (fate not incl.)'),
 ('CML v4.8 2016', 'human toxicity', 'human toxicity (HTP inf)'),
 ('CML v4.8 2016',
  'material resources: metals/minerals',
  'abiotic depletion potential (ADP): elements (ultimate reserves)'),
 ('CML v4.8 2016',
  'ozone depletion',
  'ozone layer depletion (ODP steady state)'),
 ('CML v4.8 2016',
  'photochemical oxidant formation',
  'photochemical oxidation (h

In [38]:
lca = bc.LCA(fu)
lca.lci()

results = []
for method in cml_methods:
    lca.switch_method(method)
    lca.lcia()
    results.append((method[1].title(), 
                    lca.score, 
                    bd.methods.get(method).get('unit')))
results

[('Acidification', 4262.25284277886, 'kg SO2-Eq'),
 ('Climate Change', 1344776.4563756627, 'kg CO2-Eq'),
 ('Ecotoxicity: Freshwater', 285610.5700195525, 'kg 1,4-DCB-Eq'),
 ('Ecotoxicity: Marine', 503510167.2598891, 'kg 1,4-DCB-Eq'),
 ('Ecotoxicity: Terrestrial', 4294.063843982962, 'kg 1,4-DCB-Eq'),
 ('Energy Resources: Non-Renewable', 34058665.4910202, 'megajoule'),
 ('Eutrophication', 1041.97242228296, 'kg PO4-Eq'),
 ('Human Toxicity', 101544863.0600193, 'kg 1,4-DCB-Eq'),
 ('Material Resources: Metals/Minerals', 2.090926655104789, 'kg Sb-Eq'),
 ('Ozone Depletion', 0.051422326214295405, 'kg CFC-11-Eq'),
 ('Photochemical Oxidant Formation', 396.37651616427786, 'kg ethylene-Eq')]

In [47]:
results_df = pd.DataFrame(results, columns=["Impact Category", "Score", "Unit"])
results_df = results_df.set_index("Impact Category")
results_df

,Score,Unit
Impact Category,,
Acidification,4.262253e+03,kg SO2-Eq
Climate Change,1.344776e+06,kg CO2-Eq
Ecotoxicity: Freshwater,2.856106e+05,"kg 1,4-DCB-Eq"
Ecotoxicity: Marine,5.035102e+08,"kg 1,4-DCB-Eq"
Ecotoxicity: Terrestrial,4.294064e+03,"kg 1,4-DCB-Eq"
Energy Resources: Non-Renewable,3.405867e+07,megajoule
Eutrophication,1.041972e+03,kg PO4-Eq
Human Toxicity,1.015449e+08,"kg 1,4-DCB-Eq"
Material Resources: Metals/Minerals,2.090927e+00,kg Sb-Eq
